In [1]:
!nvidia-smi 

Sun Apr 30 03:23:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Code' 
%run Unet_2D.ipynb
%run preprocessing_2D.ipynb
%run otherUnets.ipynb

torch.Size([1, 2, 192, 256])
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
66882690


In [4]:
!pip install pytorch-lightning
!pip install timm
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import timm.optim
import torch.nn as nn
from timm.data.loader import create_loader
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_isic_AC.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch", "val_loss","val_dice", "val_jac"]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

#Dataset

In [5]:
#ISIC18
data = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/ISIC2018_train.npz")
x, y = data["image"], data["mask"]
test_size = int((20/100)*x.shape[0])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=520, random_state=312)
del x, y, data

In [ ]:
#ISIC17
data_test = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/ISIC2017_192x256_test.npz")
x_test, y_test = data_test["image"], data_test["mask"]

data_train = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/ISIC2017_192x256_train.npz")
x_train, y_train = data_train["image"], data_train["mask"]

del data_test,data_train

In [ ]:
#PH2
data = np.load('/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/PH2_192_256.npz')
X_train, Y_train = data["image"], data["mask"]
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=30, random_state=312)


In [ ]:
#Tumor

data = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/brainTumorDataset.npz")
x, y = data["images"], data["masks"]
test_size = int((20/100)*x.shape[0])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=520, random_state=312)

del x, y, data

In [6]:
y_train.shape

(2074, 192, 256)

#Loss

In [7]:
columns = ['0','1']
df = pd.DataFrame(columns=columns)
_, counts =  np.unique(y_train, return_counts=True)
conts_dict = dict(zip(columns, counts))
df = df.append(conts_dict, ignore_index=True)
label_0 = df['0'].sum()
label_1 = df['1'].sum()
total_labels = label_0 + label_1
n_classes = 2
#Class weights claculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels/(n_classes*label_0)), 2)
wt1 = round((total_labels/(n_classes*label_1)), 2)
print("\n Weights are:", wt0, wt1)
# 0.63 2.37


 Weights are: 0.64 2.34


<ipython-input-7-25dbbc372c72>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(conts_dict, ignore_index=True)


In [8]:
train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True, shuffle=True, num_workers=2, drop_last=True, prefetch_factor = 8)
test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16, num_workers=2, prefetch_factor=16)

In [9]:
class SemiActiveLoss(nn.Module):
    def __init__(self, device, alpha =1e-9, beta = 1e-1, lamda = 1e-3):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.beta = beta      
        self.lamda = lamda
    def LevelsetLoss(self, image, y_pred, kernel_size=5, smooth=1e-5):
        kernel = torch.ones(1, y_pred.size(1), kernel_size, kernel_size, device=self.device) / kernel_size**2
        padding = kernel_size //2
        lossRegion = 0.0
        y_pred_fuzzy = y_pred ** 2
        for ich in range(image.size(1)):
            target_ = image[:,ich:ich+1] 
            pcentroid_local = F.conv2d(target_ * y_pred_fuzzy + smooth, kernel, padding = padding) \
                                / F.conv2d(y_pred_fuzzy + smooth, kernel, padding = padding)
            plevel_local = target_ - pcentroid_local
            loss_local = plevel_local * plevel_local * y_pred_fuzzy

            pcentroid_global = torch.sum(target_ * y_pred_fuzzy, dim=(2,3),keepdim=True) \
                                / torch.sum(y_pred_fuzzy+smooth, dim=(2,3),keepdim = True)   
            plevel_global = target_ - pcentroid_global
            loss_global = plevel_global * plevel_global * y_pred_fuzzy

            lossRegion += torch.sum(loss_local) + self.beta * torch.sum(loss_global)
        return lossRegion 
    def GradientLoss(self, y_pred, penalty = "l1"):
        dH = torch.abs(y_pred[...,1:] - y_pred[...,:-1])
        dW = torch.abs(y_pred[:,:,1:] - y_pred[:,:,:-1])
        if penalty == "l2":
            dH = dH * dH
            dW = dW * dW
        loss =  torch.sum(dH) +  torch.sum(dW)
        return loss
    def ActiveContourLoss(self, y_true, y_pred, smooth=1e-6):   
        dim = (1,2,3)
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:,1:]
        y_pred = y_pred[:,1:]

        active = - torch.log(1-y_pred+smooth) * (1-yTrueOnehot) - torch.log(y_pred+smooth) * yTrueOnehot
        loss = torch.sum(active, dim = dim) / torch.sum(yTrueOnehot + y_pred - yTrueOnehot * y_pred + smooth, dim = dim)
        return torch.mean(loss)

    def forward(self, image, y_true, y_pred):
        active = self.ActiveContourLoss(y_true, y_pred)
        # levelset =  self.LevelsetLoss(image, y_pred)
        # length = self.GradientLoss(y_pred)
        return active
        # return active + self.alpha * (levelset + self.lamda * length) 

In [10]:
class ActiveFocalLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS, gamma=2):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
        self.gamma = gamma
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)

        active_focal = - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred) \
                        - (1 - yTrueOnehot) * y_pred**self.gamma * torch.log(1 - y_pred)
        loss = torch.sum(active_focal, dim=[2, 3]) * self.class_weight
        return torch.sum(loss) / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

class ActiveContourLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)

        # y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)
        active = torch.sum(yTrueOnehot * (1 - y_pred) + (1 - yTrueOnehot) * y_pred, dim=[2, 3])
        loss = torch.sum(active * self.class_weight)
        return loss / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

In [11]:
class DiceLoss(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        intersection = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        cardinality  = torch.sum(yTrueOnehot + y_pred , dim=[1,2,3])
        loss = 1.0-torch.mean((2. * intersection + 1e-5) / (cardinality + 1e-5))
        return loss

In [12]:
class TverskyLoss(nn.Module):
    def __init__(self, device, alpha = 0.7):
        super().__init__()
        self.device = device
        self.alpha = alpha
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
        FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])
        loss = 1-torch.mean((TP + 1e-5) / (TP + self.alpha*FN + (1-self.alpha)*FP + 1e-5))
        return loss

#Training

In [13]:
%run Unet_2D.ipynb

In [14]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = Hope_Net(img_channel=3, inplane=32, dil=8, sa=True, ta=False, mhsa =True, csag= True)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score    
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss_focal = ActiveFocalLoss(self.device, [0.5, 1483.67])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.5, 1483.67])(y_true, y_pred)
        loss = loss_focal + loss_contour
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = timm.optim.Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor = 0.5, patience=16, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    

initialization done


In [15]:
segmentor = Segmentor(Hope_Net(img_channel=3, inplane=32, dil=8, sa=True, ta=False, mhsa =True, csag= True)) #Segmentor(U_Net(drop_prob=0)) 
os.makedirs('/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/tumor', exist_ok = True) 
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/tumor", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar(refresh_rate=1)
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_lrs=1e-3,swa_epoch_start=25)
PARAMS = {"accelerator": "gpu", "benchmark":True, "enable_progress_bar":True, "logger":False,
        #   "callbacks" : [progress_bar],
        #    "overfit_batches" :1,
          "callbacks" : [check_point, progress_bar, logger],
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":200,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/propose.ckpt")
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/proposed+ta_0.9173_0.8566.ckpt")


initialization done


/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [16]:
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/ckpt0.8939.ckpt")

FileNotFoundError: ignored

In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Hope_Net | 16.7 M
-----------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params
66.703    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv2d(input, weight, bias, self.stride,


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 259: 'val_dice' reached 0.75696 (best 0.75696), saving model to '/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/tumor/ckpt0.7570.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 195, global step 50764: 'val_dice' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 196, global step 51023: 'val_dice' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 197, global step 51282: 'val_dice' was not in top 1


In [ ]:
trainer.save_checkpoint("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/propose.ckpt", weights_only=True)

In [ ]:
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        layer.drop_prob = 0.05
        
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        print(layer.drop_prob) 

# segmentor.configure_optimizers()

In [ ]:
#segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False))
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/ckpt0.9101.ckpt")

In [ ]:
trainer.test(segmentor, test_dataset)

In [ ]:
weight_path = sorted(glob.glob("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/*"), reverse=True)
max_score = 0
best_weight = ""
for weight in tqdm(weight_path):
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/ckpt0.8932.ckpt") #checkpoint_path=weight
    dice_score = trainer.test(segmentor, test_dataset)[0]["test_dice"]
    clear_output()
    print(dice_score)
    if max_score < dice_score:
        max_score = dice_score
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
x_test.shape

In [ ]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
# x_inf = np.zeros((x_test.shape[0], 3, x_test.shape[1], x_test.shape[2])) #x_inf = x_test.transpose(0, -1, 1) / 255
x_inf = x_test.transpose(0, -1, 1, 2) / 255
#segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False)) #Segmentor(U_Net(drop_prob=0)) 
#segmentor = Segmentor.load_from_checkpoint(checkpoint_path="")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred = predict(x_inf, segmentor)
torch.cuda.empty_cache()
mask_predict = np.argmax(y_pred, axis=1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5) 

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))

In [ ]:
%cd /content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Visualize
np.savez_compressed("ISIC17_proposed", predicted_mask=mask_predict)

In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1, figsize=(15,15))
    plt.subplot(131), plt.imshow(x_test[i]), plt.axis('off')
    plt.subplot(132), plt.imshow(y_test[i], cmap ='gray'), plt.axis('off')
    plt.subplot(133), plt.imshow(mask_predict[i], cmap ='gray'), plt.axis('off')
    if i == 20:
        break